In [ ]:
class Simulation:
    
    '''The primary simulation class. It takes in a parameters object and runs the simulation for the 
    number of generations specified in the parameters object. It returns the population dynamics and 
    trait evolution for both species.'''

    '''It is modular and has seperate functions for ecology and trait evolution'''

    '''Ecology (which changes population number), evolution (that changes z-value are called one after the other)'''

    def __init__(self, parameters):
        '''initialse all parameters, initial population size and z-values'''
        self.parameters = parameters
        self.population_species_history_1 = [parameters.initial_population_species_1] #empty arrays for pop history and z values history'
        self.population_species_history_2 = [parameters.initial_population_species_2]
        self.z_m1_history = [parameters.z_m1_initial]
        self.z_m2_history = [parameters.z_m2_initial]
        self.z_m1 = parameters.z_m1_initial
        self.z_m2 = parameters.z_m2_initial

    def next_generation_ecology(self):
        '''takes current population size and z-values and calculates the next generation population size'''


        N1 = self.population_species_history_1[-1]  #N1 and N2 of the current generation (last val of the history arrray)
        N2 = self.population_species_history_2[-1]
      
    
        if N1 > 0.01: #next generation pop size is calculated only when current pop. size is greater than 0.01. Otherwise it is set to 0

            next_generation_competetion_term_sp1 = (self.parameters.b1 * N1) / (1 + self.parameters.alpha11 * N1 + self.parameters.alpha12 * N2)
            #ecology term for species 1 (The leslie gover equations)

            #reproductive interference term in the pop dynamics equations
            next_generation_reproductive_interference_term_sp1 = (
                    N1 * (1 + self.parameters.conspecfic_mating_rate_multiplier_sp_1 * self.z_m1)
                ) / (
                    N1 * (1 + self.parameters.conspecfic_mating_rate_multiplier_sp_1 * self.z_m1) + 
                    N2 * self.parameters.mean_heterospecifc_permissivness_females_sp_1 * (1 - self.z_m2)
                )
            next_N1 = next_generation_competetion_term_sp1*next_generation_reproductive_interference_term_sp1
            next_N1 = max(next_N1, 0)
        else:
            next_N1 = 0

        if N2 > 0.01:

            '''same logic as sp.1 '''

            next_generation_competetion_term_sp2 = (self.parameters.b2 * N2) / (
                1 + self.parameters.alpha22 * N2 + self.parameters.alpha21 * N1
            )

            next_generation_reproductive_interference_term_sp2 = (
                N2 * (1 + self.parameters.conspecfic_mating_rate_multiplier_sp_2 * self.z_m2)
            ) / (
                N2 * (1 + self.parameters.conspecfic_mating_rate_multiplier_sp_2 * self.z_m2) + 
                N1 * self.parameters.mean_heterospecifc_permissivness_females_sp_2 * (1 - self.z_m1)
            )

            next_N2 = next_generation_competetion_term_sp2 * next_generation_reproductive_interference_term_sp2

            next_N2 = max(next_N2, 0)
        else:
            next_N2 = 0

        self.population_species_history_1.append(next_N1)
        self.population_species_history_2.append(next_N2)

    def next_generation_traits(self, N1, N2):

        '''takes current population size and z-values and calculates the next generation z-values'''

        if N1 > 0.01: #Once again, the next generation is relevant only when the current population size is greater than 0.01
            delta_z_m1 = self.parameters.V_A1 * (self.parameters.conspecfic_mating_rate_multiplier_sp_1) / (1 + self.parameters.conspecfic_mating_rate_multiplier_sp_1*self.z_m1) #Change in z-value (refer to the doc)
            new_z_m1 = self.z_m1 + delta_z_m1 #new z-value
            
            if new_z_m1 <= 1:  #z-value is capped at 1. So the update is valid only when the new z-value is less than or equal to 1
                self.z_m1 = new_z_m1
            else:
                self.z_m1 = 1  #if the new z-value is greater than 1, it is set to 1

            self.z_m1_history.append(self.z_m1)

        if N2 > 0.01:
            delta_z_m2 = self.parameters.V_A2 * (self.parameters.conspecfic_mating_rate_multiplier_sp_2) / (1 + self.parameters.conspecfic_mating_rate_multiplier_sp_2*self.z_m2)

            new_z_m2 = self.z_m2 + delta_z_m2
            
            if new_z_m2 <= 1:
                self.z_m2 = new_z_m2
            else:
                self.z_m2 = 1

            self.z_m2_history.append(self.z_m2)
            
    def next_generation(self):
        '''runs the ecology and evolution functions for the next generation. Note that there is a seperation of timescales. (Model is in discrete time). 
        But this does not influene rresults)'''
        self.next_generation_ecology()
        N1 = self.population_species_history_1[-1]
        N2 = self.population_species_history_2[-1]
        self.next_generation_traits(N1, N2)
        return N1, N2

    def run_simulation(self):
        #run simulaton for the number of generations we define in the parameters object

        for t in range(self.parameters.num_generations - 1):
            N1, N2 = self.next_generation()
            if N1 < 0.01 or N2 < 0.01:
                break

        return self.population_species_history_1, self.population_species_history_2, self.z_m1_history, self.z_m2_history